# Used Code for the Thesis:

# Dark Energy: Constraining Cosmological Models with Extra Spatial Dimensions Based on Type Ia Supernovae Data

## $w$CDM Model

### Fitting the Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Funktion zum Laden der Pantheon+SH0ES-Daten
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, sep=' ')
    z = np.array(dat['zHD'])  # Redshifts
    dm = np.array(dat['MU_SH0ES'])  # Distanzmoduli

    ncol = 1701
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    selnotlocal = z > 0.01
    select = selnotlocal

    z = z[select]
    dm = dm[select]
    thecov = cov1[select, :]
    thecov = thecov[:, select]
    
    return z, dm, thecov

# Funktion zur Berechnung der Leuchtkraftentfernung im wCDM-Modell
def luminosity_distance(z, omegam, h0, w=-1.0):
    c = 299792.458  # Lichtgeschwindigkeit in km/s

    def e(z):
        return 1.0 / np.sqrt(omegam * (1 + z)**3 + (1 - omegam) * (1 + z)**(3 * (1 + w)))

    integral = np.array([
        np.trapz(e(np.linspace(0, zi, 200)), np.linspace(0, zi, 200)) for zi in z
    ])

    d_L = (c / h0) * (1 + z) * integral
    return d_L

# Modellfunktion für das Distanzmodul
def model_func(z, omegam, h0, w=-1.0):
    d_L = luminosity_distance(np.array([z]), omegam, h0, w)[0]
    return 5 * np.log10(d_L) + 25

# Chi²-Funktion (nur w wird gefittet, Omega_m und H0 fest)
def chi_squared_fixed_omega_h0(w, z, dm, cov_matrix, omegam_fixed, h0_fixed):
    mag_model = np.array([model_func(zi, omegam_fixed, h0_fixed, w[0]) for zi in z])
    delta_m = dm - mag_model
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    chi2 = delta_m.T @ inv_cov_matrix @ delta_m
    return chi2

# Chi²-Funktion für freien Fit von Omega_m, H0, w
def chi_squared_all(params, z, dm, cov_matrix):
    omegam, h0, w = params
    mag_model = np.array([model_func(zi, omegam, h0, w) for zi in z])
    delta_m = dm - mag_model
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    chi2 = delta_m.T @ inv_cov_matrix @ delta_m
    return chi2

# Fit-Funktion für w bei festen Omega_m und H0
def fit_w_fixed_omega_h0(z, dm, cov_matrix, omegam_fixed, h0_fixed):
    result = minimize(
        chi_squared_fixed_omega_h0,
        x0=[-1.0],
        bounds=[(-2.0, 0.0)],
        args=(z, dm, cov_matrix, omegam_fixed, h0_fixed)
    )
    return result.x[0]

# Fit-Funktion für alle drei Parameter
def fit_all_parameters(z, dm, cov_matrix):
    result = minimize(
        chi_squared_all,
        x0=[0.3, 70.0, -1.0],
        bounds=[(0.1, 1.0), (50.0, 90.0), (-2.0, 0.0)],
        args=(z, dm, cov_matrix)
    )
    return result.x  # Rückgabe: [Omega_m, H0, w]

# ---- Hauptcode ----

# Pfade zu den Dateien (anpassen!)
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"

# Daten einlesen
z_pantheon, dm_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)
dm_errors = np.sqrt(np.diag(cov_matrix_pantheon))

# === Freier Fit für Omega_m, H0, w ===
omega_m_best, h0_best, w_best = fit_all_parameters(z_pantheon, dm_pantheon, cov_matrix_pantheon)
print(f"[Freier Fit] Omega_M = {omega_m_best:.4f}, H_0 = {h0_best:.4f}, w = {w_best:.4f}")

# === Plot ===
z_fit_values = np.linspace(min(z_pantheon), max(z_pantheon), 300)

dm_fit_free = [model_func(zf, omega_m_best, h0_best, w_best) for zf in z_fit_values]

plt.figure(figsize=(10, 6))
plt.errorbar(z_pantheon, dm_pantheon, yerr=dm_errors, fmt='o', color='red', label='Pantheon+SHOES Data')
plt.plot(z_fit_values, dm_fit_free, 'b-', label=f"Frei: $w$={w_best:.3f}, $\Omega_m$={omega_m_best:.3f}, $H_0$={h0_best:.3f}")

plt.xlabel("Redshift $z$")
plt.ylabel("Relative Magnitude $m_{rel}$")
plt.title(r"Pantheon+SHOES: Relative Magnitude vs. Redshift Fit ($w$CDM Model)")
plt.savefig('m_rel_vs_z_fit_wCDM', dpi=300)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig('wcdm_fit.png', dpi=300)
plt.show()

### Contour Plot for Uncertainies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import quad, simps
from scipy.optimize import fsolve
from matplotlib.lines import Line2D
from tqdm import tqdm
import os

# --- Konstanten ---
c = 299792.458  # Lichtgeschwindigkeit in km/s

# --- Fehlerbehandlung ---
def safe_quad(integrand, a, b, args=()):
    try:
        result, _ = quad(integrand, a, b, args=args, limit=100)
        if np.isnan(result) or np.isinf(result):
            raise ValueError("quad returned NaN or Inf")
        return result
    except Exception as e:
        print(f"Fehler bei quad: {e}")
        return np.nan

# --- H0-freie Luminositätsentfernung im wCDM-Modell ---
def e_inv(z, omegam, w):
    return 1.0 / np.sqrt(omegam * (1 + z)**3 + (1 - omegam) * (1 + z)**(3 * (1 + w)))

def d_L_wCDM(z, omegam, w):
    try:
        integral = safe_quad(e_inv, 0, z, args=(omegam, w))
        if np.isnan(integral):
            return np.nan
        return c * (1 + z) * integral  # in Mpc (ohne H0)
    except:
        return np.nan

# --- Marginalisiertes χ² über 𝓜 ---
def marginalized_chi2_wCDM(z, m_obs, m_err, omegam, w):
    d_Ls = np.array([d_L_wCDM(zi, omegam, w) for zi in z])
    if np.any(np.isnan(d_Ls)) or np.any(d_Ls <= 0):
        return np.inf

    log_d_Ls = 5 * np.log10(d_Ls)

    M_vals = np.linspace(15, 16.5, 200)
    chi2_vals = []

    for M in M_vals:
        model_mags = M + log_d_Ls
        chi2 = np.sum(((m_obs - model_mags) / m_err) ** 2)
        chi2_vals.append(np.exp(-0.5 * chi2))

    integral = simps(chi2_vals, M_vals)
    if integral <= 0 or not np.isfinite(integral):
        return np.inf

    chi2_marg = -2 * np.log(integral)
    return chi2_marg

# --- Daten einlesen ---
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, delim_whitespace=True)
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])

    ncol = len(dat)
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    select = z > 0.01
    z = z[select]
    dm = dm[select]
    thecov = cov1[select, :]
    thecov = thecov[:, select]
    return z, dm, thecov

# --- Gitterberechnung für wCDM ---
def run_large_grid_wCDM(omega_m_vals, w_vals, z_pantheon, mu_pantheon, mu_errors, checkpoint_interval=10):
    chi2_grid = np.zeros((len(omega_m_vals), len(w_vals)))

    for i, Omega_m in tqdm(enumerate(omega_m_vals), total=len(omega_m_vals), desc="Berechne χ²-Gitter (wCDM)"):
        for j, w in enumerate(w_vals):
            try:
                chi2 = marginalized_chi2_wCDM(z_pantheon, mu_pantheon, mu_errors, Omega_m, w)
                chi2_grid[i, j] = chi2
            except Exception as e:
                print(f"Fehler bei Omega_m={Omega_m}, w={w}: {e}")
                chi2_grid[i, j] = np.inf

            if (i * len(w_vals) + j) % checkpoint_interval == 0:
                np.save("chi2_grid_wCDM_checkpoint.npy", chi2_grid)

    return chi2_grid

# --- Hauptprogramm ---

# Pfade zu Pantheon+SH0ES-Daten (ggf. anpassen!)
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"

# Daten laden
z_pantheon, mu_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)
mu_errors = np.sqrt(np.diag(cov_matrix_pantheon))

# Parameter-Gitter
omega_m_vals = np.linspace(0.1, 0.6, 170)
w_vals = np.linspace(-3.0, 1, 170)

# χ²-Gitter berechnen
chi2_grid_wCDM = run_large_grid_wCDM(omega_m_vals, w_vals, z_pantheon, mu_pantheon, mu_errors)

# Gitter speichern
np.save("chi2_grid_wCDM.npy", chi2_grid_wCDM)
np.save("omega_m_vals.npy", omega_m_vals)
np.save("w_vals.npy", w_vals)
print("Gitter gespeichert.")

# --- Plot erstellen ---
# --- Gespeicherte Gitterdaten laden ---
chi2_grid = np.load("chi2_grid_wCDM.npy")
omega_m_vals = np.load("omega_m_vals.npy")
w_vals = np.load("w_vals.npy")

# --- Mindestwert für Chi² extrahieren ---
min_chi2 = np.nanmin(chi2_grid)

# --- Contour levels (1σ, 2σ, 3σ) ohne dof-Korrektur ---
contour_levels = [min_chi2 + delta for delta in [2.30, 6.17, 11.8]]

# --- Relevanter Bereich zur Skalierung der Achsen ---
relevant_indices = np.where((chi2_grid >= contour_levels[0]) & (chi2_grid <= contour_levels[-1]))

if relevant_indices[0].size > 0 and relevant_indices[1].size > 0:
    min_w = w_vals[relevant_indices[1]].min()
    max_w = w_vals[relevant_indices[1]].max()
    min_omega_m = omega_m_vals[relevant_indices[0]].min()
    max_omega_m = omega_m_vals[relevant_indices[0]].max()
else:
    min_w, max_w = w_vals.min(), w_vals.max()
    min_omega_m, max_omega_m = omega_m_vals.min(), omega_m_vals.max()

# --- Plot erstellen ---
plt.figure(figsize=(10, 8))
plt.rcParams['axes.facecolor'] = 'white'
plt.gcf().set_facecolor('white')

# Meshgrid für Konturen
W_mesh, Omega_m_mesh = np.meshgrid(w_vals, omega_m_vals)

# Farben und Labels für die Konturen
contour_colors = ['blue', 'green', 'red']
labels = [
    r'1$\sigma$ (Δ$\chi^2$=2.30)',
    r'2$\sigma$ (Δ$\chi^2$=6.17)',
    r'3$\sigma$ (Δ$\chi^2$=11.8)'
]

# Konturen plotten
lines = []
for level, color in zip(contour_levels, contour_colors):
    contour = plt.contour(W_mesh, Omega_m_mesh, chi2_grid,
                          levels=[level], colors=[color], linewidths=2)
    lines.append(Line2D([0], [0], color=color, lw=2))


# Achsentitel und Beschriftung
plt.xlabel(r'$w$', fontsize=14)
plt.ylabel(r'$\Omega_m$', fontsize=14)
plt.title(r'$\chi^2$ Contours for $\Omega_m$ and $w$', fontsize=18)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Dynamische Achsenskalierung
plt.xlim(min_w - 0.05, max_w + 0.05)
plt.ylim(min_omega_m - 0.01, max_omega_m + 0.01)

plt.tight_layout()
plt.savefig('chi2_contours_wCDM_thesis.png', dpi=300)
plt.show()

# --- χ²-minimierende Werte ausgeben ---
min_chi2_index = np.unravel_index(np.nanargmin(chi2_grid_wCDM), chi2_grid_wCDM.shape)
best_omega_m = omega_m_vals[min_chi2_index[0]]
best_w = w_vals[min_chi2_index[1]]

print(f"Die minimalen χ²-Werte treten bei Omega_m = {best_omega_m:.4f} und w = {best_w:.4f} auf.")

### MCMC Contour Plots

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from scipy.integrate import quad
import pandas as pd
import emcee
from matplotlib.lines import Line2D

# --- Cosmological constants ---
c = 299792.458  # speed of light in km/s

# --- New model function with w ---
def e(z, omegam, w):
    return 1.0 / np.sqrt(omegam * (1 + z)**3 + (1 - omegam) * (1 + z)**(3 * (1 + w)))

def luminosity_distance(z, omegam, h0, w):
    integral, _ = quad(e, 0, z, args=(omegam, w))
    d_L = (c / h0) * (1 + z) * integral
    return d_L

def model_func(z, omegam, h0, w):
    d_L = luminosity_distance(z, omegam, h0, w)
    return 5 * np.log10(d_L) + 25

# --- Likelihood functions ---
def log_likelihood(params, z, dm, inv_cov):
    omegam, h0, w = params
    mag_model = np.array([model_func(zi, omegam, h0, w) for zi in z])
    delta_m = dm - mag_model
    chi2 = delta_m.T @ inv_cov @ delta_m
    return -0.5 * chi2, chi2

def log_prior(params):
    omegam, h0, w = params
    if 0.1 < omegam < 0.999 and 50 < h0 < 90 and -3.0 < w < 1.0:
        return 0.0
    return -np.inf

def log_posterior(params, z, dm, inv_cov):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf, np.inf
    ll, chi2 = log_likelihood(params, z, dm, inv_cov)
    return lp + ll, chi2

# --- Data loading ---
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, delim_whitespace=True)
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])
    ncol = len(dat)
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)
    select = z > 0.01
    return z[select], dm[select], cov1[select][:, select]

# --- File paths ---
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"

# --- Load data ---
z_pantheon, mu_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)

# --- Compute the inverse of the covariance matrix ---
inv_cov_matrix_pantheon = np.linalg.inv(cov_matrix_pantheon)

# --- MCMC setup ---
ndim = 3  # Omega_m, H_0, w
nwalkers = 30
nsteps = 5000
initial_guess = [0.3, 73.0, -1.0]
pos = initial_guess + 1e-1 * np.random.randn(nwalkers, ndim)

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior,
                                 args=(z_pantheon, mu_pantheon, inv_cov_matrix_pantheon))
sampler.run_mcmc(pos, nsteps, progress=True)

samples = sampler.get_chain(discard=1000, flat=True)
Omega_m_samples, H_0_samples, w_samples = samples[:, 0], samples[:, 1], samples[:, 2]

# --- Best-fit parameters ---
log_likelihoods = np.array([log_likelihood(params, z_pantheon, mu_pantheon, inv_cov_matrix_pantheon) for params in samples])
best_index = np.argmax(log_likelihoods[:, 0])
best_params = samples[best_index]
best_Omega_m, best_H_0, best_w = best_params
chi2_min = -2 * log_likelihoods[best_index, 0]

# --- 2D Histogram for (w, Omega_m) ---
nbins = 70
w_range = (-3.0, 1.0)
omegam_range = (0.01, 0.8)

# Compute the density for Omega_m and w
H, xedges, yedges = np.histogram2d(w_samples, Omega_m_samples, bins=nbins, range=[w_range, omegam_range])
H_smooth = gaussian_filter(H, sigma=1.0)

xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
X, Y = np.meshgrid(xbin, ybin)

# Calculate contour levels for the density of w and Omega_m
sorted_H = np.sort(H_smooth.flatten())[::-1]
cumulative = np.cumsum(sorted_H)
cumulative /= cumulative[-1]
cred_levels = [0.683, 0.954, 0.997]
contour_levels = [sorted_H[np.searchsorted(cumulative, level)] for level in cred_levels]
contour_levels = np.sort(contour_levels)

# --- Plot ---
plt.figure(figsize=(11, 8))

# Contour plot filling
plt.contourf(X, Y, H_smooth.T, levels=50, cmap='Blues', alpha=0.7)

# Contour lines for the credible regions
contour_colors = ['red', 'green', 'blue']
labels = [r'99.7% credible region (3$\sigma$)', r'95.4% credible region (2$\sigma$)', r'68.3% credible region (1$\sigma$)']
lines = []
for level, color in zip(contour_levels, contour_colors):
    line = plt.contour(X, Y, H_smooth.T, levels=[level], colors=[color], linewidths=2)
    lines.append(line.legend_elements()[0][0])

# Best-fit parameters label
best_label = fr'Best Fit: $\Omega_m = {best_Omega_m:.3f}$, $w = {best_w:.3f}$'
dummy_line = Line2D([0], [0], color='white', alpha=0)
lines.append(dummy_line)
labels.append(best_label)

plt.legend(lines, labels, loc='upper right', fontsize=12)
plt.title(r'Contour Plots of $\Omega_m$ and $w$ (5000 Steps Total)', fontsize=18)
plt.xlabel(r'$w$', fontsize=14)
plt.ylabel(r'$\Omega_m$', fontsize=14)
plt.xlim(w_range)
plt.ylim(omegam_range)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("contour_plot_MCMC_Omegam_w.png", dpi=300)
plt.show()

# --- Save results ---
np.savez("mcmc_results_w_Omegam.npz",
         Omega_m_samples=Omega_m_samples,
         H_0_samples=H_0_samples,
         w_samples=w_samples,
         best_params=best_params,
         chi2_min=chi2_min,
         walker_positions=np.array(pos),
         chain=sampler.get_chain(),
         lnprobability=sampler.get_log_prob())

### Histograms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.integrate import quad
from tqdm import tqdm

# --------------------------------------------
# Daten einlesen & vorbereiten
# --------------------------------------------

# Lade Pantheon+SH0ES-Daten
dat = pd.read_csv('/Users/insertdatapath/Pantheon+SH0ES.dat', sep=' ')
z = np.array(dat['zHD'])
dm = np.array(dat['MU_SH0ES'])

# Lade Kovarianzmatrix
ncol = 1701
cov1 = np.loadtxt("/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov")
cov1.shape = (ncol, ncol)
thecov = cov1

# Filtere lokale Supernovae (nur z > 0.01)
select = z > 0.01
z_filtered = z[select]
dm_filtered = dm[select]
thecov_filtered = thecov[select, :][:, select]

# Für später
z = z_filtered
dm = dm_filtered
cov_matrix = thecov_filtered

# --------------------------------------------
# Kosmologie: wCDM-Modellfunktionen
# --------------------------------------------

c = 299792.458  # Lichtgeschwindigkeit in km/s

def H_wCDM(z, Omega_m, H_0, w):
    E_z = np.sqrt(Omega_m * (1 + z)**3 + (1 - Omega_m) * (1 + z)**(3 * (1 + w)))
    return H_0 * E_z

def d_L_wCDM(z, Omega_m, H_0, w):
    integrand = lambda ztilde: 1.0 / H_wCDM(ztilde, Omega_m, H_0, w)
    integral, _ = quad(integrand, 0, z, limit=100)
    return c * (1 + z) * integral

def model_func_wCDM(z_array, Omega_m, H_0, w):
    return 5 * np.log10([d_L_wCDM(zi, Omega_m, H_0, w) for zi in z_array]) + 25

# --------------------------------------------
# Chi²-Funktion
# --------------------------------------------

inv_cov_matrix = np.linalg.inv(cov_matrix)

def chi_squared_wCDM(params, z, dm, inv_cov_matrix):
    Omega_m, H_0, w = params
    mag_model = model_func_wCDM(z, Omega_m, H_0, w)
    delta_m = dm - mag_model
    return delta_m.T @ inv_cov_matrix @ delta_m

# --------------------------------------------
# Monte Carlo Simulation
# --------------------------------------------

def simulate_supernovae(z, Omega_m, H_0, w, cov_matrix):
    dm_model = model_func_wCDM(z, Omega_m, H_0, w)
    error = np.random.multivariate_normal(np.zeros(len(z)), cov_matrix)
    dm_sim = dm_model + error
    return dm_sim, cov_matrix

# Startparameter
Omega_m_true = 0.2929
H0_true = 73.1281
w_true = -0.9067
n_simulations = 1500
#Omega_M = 0.2929, H_0 = 73.1281, w = -0.9067

# Ergebnisse speichern
fitted_omegam = []
fitted_h0 = []
fitted_w = []

# Monte Carlo Simulation starten
print(f"Starte Monte Carlo Simulation für wCDM-Modell (Ωm={Omega_m_true:.3f}, H0={H0_true:.2f}, w={w_true})")

for _ in tqdm(range(n_simulations)):
    dm_sim, cov_sim = simulate_supernovae(z, Omega_m_true, H0_true, w_true, cov_matrix)

    result = minimize(
        chi_squared_wCDM,
        x0=[0.3, 70, -1.0],  # Startwerte für [Omega_m, H0, w]
        args=(z, dm_sim, cov_sim),
        bounds=[(0.1, 0.6), (50, 90), (-3, 1)],  
        method='L-BFGS-B'
    )

    if result.success:
        omega_fit, h0_fit, w_fit = result.x
        fitted_omegam.append(omega_fit)
        fitted_h0.append(h0_fit)
        fitted_w.append(w_fit)

# --------------------------------------------
# Ergebnisse visualisieren
# --------------------------------------------

fitted_omegam = np.array(fitted_omegam)
fitted_h0 = np.array(fitted_h0)
fitted_w = np.array(fitted_w)
plt.figure(figsize=(18, 5))

# Ωm Histogramm
plt.subplot(1, 3, 1)
plt.hist(fitted_omegam, bins=30, color='skyblue', edgecolor='black')
plt.axvline(np.mean(fitted_omegam), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_omegam):.3f}")
plt.xlabel(r'Fitted $\Omega_m$')
plt.ylabel('Incidence')
plt.title(r'Distribution of $\Omega_m$')
plt.legend()

# H0 Histogramm
plt.subplot(1, 3, 2)
plt.hist(fitted_h0, bins=30, color='lightgreen', edgecolor='black')
plt.axvline(np.mean(fitted_h0), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_h0):.2f}")
plt.xlabel(r'Fitted $H_0$')
plt.ylabel('Incidence')
plt.title('Distribution of $H_0$')
plt.legend()

# w Histogramm
plt.subplot(1, 3, 3)
plt.hist(fitted_w, bins=30, color='orange', edgecolor='black')
plt.axvline(np.mean(fitted_w), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_w):.2f}")
plt.xlabel(r'Fitted $w$')
plt.ylabel('Incidence')
plt.title(r'Distribution of $w$')
plt.legend()

plt.tight_layout()
plt.show()

# --------------------------------------------
# Ergebnisse speichern
# --------------------------------------------

np.savez("montecarlo_histo_wCDM_results.npz",
         fitted_omegam=fitted_omegam,
         fitted_h0=fitted_h0,
         fitted_w=fitted_w,
         z=z,
         dm=dm,
         cov_matrix=cov_matrix,
         Omega_m_true=Omega_m_true,
         H0_true=H0_true,
         w_true=w_true,
         n_simulations=n_simulations)

print("Simulationsergebnisse gespeichert")